In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import wget
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, f1_score,confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from wordcloud import WordCloud

In [ ]:
# wget.download("https://lazyprogrammer.me/course_files/spam.csv")

In [ ]:
df=pd.read_csv('datasets/spam.csv', encoding='ISO-8859-1') #encoding coz of some characters that are not valid in utf-8 encoding

In [ ]:
df.head()

In [ ]:
df=df.drop(["Unnamed: 2","Unnamed: 3","Unnamed: 4"],axis=1)

In [ ]:
df.head()

In [ ]:
df.columns=['labels','data']

In [ ]:
df.head()

In [ ]:
df['labels'].hist()

In [ ]:
df['b_labels']=df['labels'].map({'ham':0,'spam':1})
Y=df['b_labels'].to_numpy()

In [ ]:
#split data
X_train,X_test,y_train,y_test=train_test_split(df['data'],Y,test_size=0.33)

In [ ]:
# featurizer=TfidfVectorizer(decode_error='ignore')
# X_train=featurizer.fit_transform(X_train)
# X_test=featurizer.transform(X_test)

featurizer=CountVectorizer(decode_error='ignore')
X_train=featurizer.fit_transform(X_train)
X_test=featurizer.transform(X_test)

In [ ]:
X_train

In [ ]:
#create model,train it, print score
model=MultinomialNB()
model.fit(X_train,y_train)
print("Train acc: ",model.score(X_train,y_train))
print("Test acc: ",model.score(X_test,y_test))

In [ ]:
p_train=model.predict(X_train)
p_test=model.predict(X_test)
print("Train F1-score: ",f1_score(y_train,p_train))
print("Test F1-score: ",f1_score(y_test,p_test))

In [ ]:
prob_train=model.predict_proba(X_train)[:,1]
prob_test=model.predict_proba(X_test)[:,1]
print("train AUC: ",roc_auc_score(y_train,prob_train))
print("test AUC: ",roc_auc_score(y_test,prob_test))

In [ ]:
cm=confusion_matrix(y_train,p_train)
cm

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=['ham','spam'])
disp.plot()

In [ ]:
cm_test=confusion_matrix(p_test,y_test)
cm_test

In [ ]:
disp = ConfusionMatrixDisplay(confusion_matrix=cm_test, display_labels=['ham','spam'])
disp.plot()

In [ ]:
#visualize data
def visualize(label):
    words=''
    for msg in df[df['labels']==label]['data']:
        msg=msg.lower()
        words+=msg
    wordcloud=WordCloud(width=600,height=400).generate(words)
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

In [ ]:
visualize('spam')

In [ ]:
visualize('ham')

In [ ]:
#see what we are getting wrong
X=featurizer.transform(df['data'])
df['predictions']=model.predict(X)

In [ ]:
#things that should be spam
sneaky_spam=df[(df['predictions']==0) & (df['b_labels']==1)]['data']
for msg in sneaky_spam:
    print(msg)

In [ ]:
#not actually spam
not_actually_spam=df[(df['predictions']==1) & (df['b_labels']==0)]['data']
for msg in not_actually_spam:
    print(msg)